In [ ]:
import cv2

def detect_faces(image_path, output_dir):
    # Load the image
    image = cv2.imread(image_path)
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Load the cascade classifier
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    # Iterate over the detected faces
    for i, (x, y, w, h) in enumerate(faces):
        # Crop the face and save it to the specified directory
        headshot = image[y:y+h, x:x+w]
        cv2.imwrite(output_dir + '/headshot_' + str(i) + '.jpg', headshot)
    return len(faces)

# Example usage
num_of_faces = detect_faces('face_detection_2x.png', 'output')
print("number of detected faces", num_of_faces)


In [ ]:
import boto3
from PIL import Image

def copy_images(src_bucket_name, dst_bucket_name):
    # Create S3 client
    s3 = boto3.client('s3')
    # Get a list of all image files in the source bucket
    result = s3.list_objects(Bucket=src_bucket_name)
    image_files = [content['Key'] for content in result.get('Contents', []) if content['Key'].endswith(('jpg','png','jpeg'))]

    # Iterate over the image files
    for image_file in image_files:
        # Download the image file from the source bucket
        s3.download_file(src_bucket_name, image_file, '/tmp/'+image_file)
        # Open the image file with PIL
        with Image.open('/tmp/'+image_file) as img:
            # Check if the image file has transparent pixels
            if img.mode in ('RGBA', 'LA'):
                # Open a file to log the image files with transparent pixels
                with open("transparent_images.txt", "a") as f:
                    f.write(image_file + '\n')
            else:
                # Copy the image file to the destination bucket
                s3.upload_file('/tmp/'+image_file, dst_bucket_name, image_file)

# Example usage
copy_images('src-bucket-name', 'dst-bucket-name')
